In [1]:
import json
with open('./kaggle.json', 'w') as f:
    json.dump({"username":"jmorlandbarrett","key":"d6f20f224b408dfa4b9350e90e129dd9"}, f)
import opendatasets as od
import pandas as pd
import os
from tqdm import tqdm
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
from matplotlib import dates as mdates
import pytz
from scipy import stats
import plotly.express as px

od.download("https://www.kaggle.com/datasets/ranakrc/smart-building-system")

100%|██████████| 74.7M/74.7M [00:12<00:00, 6.17MB/s]


In [2]:
temporal_df = pd.DataFrame()

for folder, _, files in os.walk('./smart-building-system/KETI'): #iterate each folder from dataset
        if folder == './smart-building-system/KETI': #skip root folder as it only contains the readme file
            continue
        room_df = pd.DataFrame()
        room = folder.split('/')[-1]
        room = room[5:]
        for i, file in enumerate(files):
            feat_name = file.split('.')[0] # get the name of the sensor e.g. CO2
            tmp_df = pd.read_csv(os.path.join(folder, file), names=['TIME', feat_name], header=0)    
            tmp_df.sort_values(by=['TIME'], ascending=False,inplace=True) #sort by time values
            tmp_df = tmp_df.set_index('TIME') #set the time value as the index for the dataframe so that concat works on same index values.
                        
            room_df = tmp_df if i == 0 else pd.concat([room_df, tmp_df], axis=1) #concatonate sensors with the same time value
            room_df.fillna(method='ffill', inplace=True) #forward fill missing numbers e.g. inbetween samples assume that the previous state hasn't changed.
              
        room_df['ROOM'] = room
        room_df['FLOOR'] = int(room[0])
        temporal_df = pd.concat([temporal_df, room_df], axis=0) #join the current room dataframe to the total dataframe

In [3]:
tz = pytz.timezone('US/Pacific')
temporal_df.reset_index(inplace=True)
temporal_df['TIME'] = [datetime.fromtimestamp(x,tz=tz) for x in temporal_df['TIME']]
temporal_df['TIME'] = pd.to_datetime(temporal_df['TIME'])
temporal_df['pir'].fillna(method='bfill', inplace=True)
temporal_df['pir'] = temporal_df['pir'].astype('bool')

In [4]:
pd.set_option('float_format', '{:f}'.format)
temporal_df.loc[(temporal_df['temperature'] > 100.00) | (temporal_df['temperature'] < 0), ['temperature']] = np.nan
temporal_df['temperature'].fillna(method='ffill', inplace=True)
temporal_df['humidity'].fillna(method='ffill', inplace=True)
temporal_df['light'].fillna(method='ffill', inplace=True)
temporal_df.loc[(temporal_df['humidity'] < 0.00), ['humidity']] = np.nan
temporal_df['humidity'].fillna(method='ffill', inplace=True)
temporal_df.loc[(temporal_df['light'] > 1000.00), ['light']] = np.nan
temporal_df['light'].fillna(method='ffill', inplace=True)

In [5]:
#general cleanup
del feat_name, file, files, folder, room, room_df, tmp_df, i, f

In [6]:
df4 = temporal_df[(temporal_df['ROOM'].isin(['413']))].groupby([pd.Grouper(key="TIME",freq='5min')]).agg({
    'temperature':'mean',
    'humidity':'mean',
    'light':"mean",
    'co2':"mean",
    'pir':'first'})
df4=df4.reset_index()
df4 = df4.set_index("TIME")
#df4=df4.drop(['ROOM','FLOOR'], axis=1)

In [7]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

initial_range = [
    '2013-08-28 08:00:00', '2013-08-28 20:00:00'
]

for room in temporal_df['ROOM'].unique():
    df4 = temporal_df[(temporal_df['ROOM'].isin([room]))].groupby([pd.Grouper(key="TIME",freq='5min')]).agg({
        'temperature':'mean',
        'humidity':'mean',
        'light':"mean",
        'co2':"mean",
        'pir':'first'})
    df4=df4.reset_index()
    df4 = df4.set_index("TIME")
    subfig = make_subplots(specs=[[{"secondary_y": True}]])

    fig1 = go.Bar(
            x = df4.index,
            y = df4['pir'],
            name = 'pir',
            opacity=0.5,
            marker_color='firebrick'
        )


    for column in df4.columns.to_list():
        vis= False
        if column == 'pir':
            continue
        if column == 'co2':
            vis = True
        subfig.add_trace(
            go.Scatter(
                x = df4.index,
                y = df4[column],
                name = column,
                visible=vis
            ),
        secondary_y=True,
        )
    subfig.add_trace(
        fig1,
        secondary_y=False,
    )
    subfig.update_layout(
        updatemenus=[go.layout.Updatemenu(
            active=0,
            buttons=list(
                [dict(label = 'CO2',
                    method = 'update',
                    args = [{'visible': [False, False, False, True, True]},
                            {'title': 'Co2 exploration for room '+room,
                            'showlegend':True,
                            'yaxis2.title':'Co2'}]),
                dict(label = 'Temperature',
                    method = 'update',
                    args = [{'visible': [True, False, False, False, True]}, # the index of True aligns with the indices of plot traces
                            {'title': 'Temperature exploration for room '+room,
                            'showlegend':True,
                            'yaxis2.title':'Temperature(c)'}]),
                dict(label = 'Humidity',
                    method = 'update',
                    args = [{'visible': [False, True, False, False, True]},
                            {'title': 'Humidity exploration for room '+room,
                            'showlegend':True,
                            'yaxis2.title':'Humidity(%)'}]),
                dict(label = 'Light',
                    method = 'update',
                    args = [{'visible': [False, False, True, False, True]},
                            {'title': 'Light exploration for room '+room,
                            'showlegend':True,
                            'yaxis2.title':'Light(lux)'}]),
                dict(label = 'All',
                    method = 'update',
                    args = [{'visible': [True, True, True, True, True]},
                            {'title': 'All variable exploration for room '+room,
                            'showlegend':True,
                            'yaxis2.title':'All'}])
                ])
            )
        ])

    subfig.layout.yaxis2.title='CO2'
    subfig.layout.title='Co2 exploration for room '+room
    subfig.layout.yaxis.title=None
    subfig.layout.yaxis.showticklabels=False
    subfig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1h", step="hour", stepmode="todate"),
                dict(count=6, label="6h", step="hour", stepmode="todate"),
                dict(count=12, label="12h", step="hour", stepmode="todate"),
                dict(count=1, label="1d", step="day", stepmode="todate"),
                dict(step="all")
            ])
        )
    )
    subfig.layout.xaxis.update(range=initial_range)
    subfig.write_html("..\\Interactive System\\pir-exploration\\"+room+".html")
subfig.show()



## Part 2 - Visualising Floor Temperatures

In [8]:
#clean up
del column, fig1,initial_range,room,df4,subfig,tz,vis

In [9]:
print("Number of rooms per floor:")
print("4:",temporal_df[temporal_df['FLOOR']==4]['ROOM'].nunique())
print("5:",temporal_df[temporal_df['FLOOR']==5]['ROOM'].nunique())
print("6:",temporal_df[temporal_df['FLOOR']==6]['ROOM'].nunique())
print("7:",temporal_df[temporal_df['FLOOR']==7]['ROOM'].nunique())

Number of rooms per floor:
4: 16
5: 9
6: 13
7: 13


In [10]:
temporal_df[temporal_df['FLOOR']==5]['ROOM'].unique()

array(['510', '511', '513', '552', '554', '556', '558', '562', '564'],
      dtype=object)

In [11]:
df5 = temporal_df[(temporal_df['FLOOR']==5)].groupby(['ROOM',pd.Grouper(key="TIME",freq='15min')]).agg({
        'temperature':'mean'})
df5 = df5.reset_index()
#df5 = df5.set_index('TIME')

In [12]:
df5.loc[df5['ROOM']=='510', ['x','y']] = 4,36
df5.loc[df5['ROOM']=='511', ['x','y']] = 4,45
df5.loc[df5['ROOM']=='513', ['x','y']] = 12,46
df5.loc[df5['ROOM']=='552', ['x','y']] = 45,4
df5.loc[df5['ROOM']=='554', ['x','y']] = 55,4
df5.loc[df5['ROOM']=='556', ['x','y']] = 64.5,4
df5.loc[df5['ROOM']=='558', ['x','y']] = 70.5,4
df5.loc[df5['ROOM']=='562', ['x','y']] = 76.5,4
df5.loc[df5['ROOM']=='564', ['x','y']] = 83.5,4

In [13]:
from matplotlib import markers
import plotly.express as px
fig = px.scatter(
    df5,
    x="x",
    y="y",
    animation_group="ROOM",
    size="temperature",
    color="temperature",
    animation_frame="TIME",
    size_max=45,
    width=1200,
    height=500,
    hover_data={'temperature':True,
                "TIME":False,
                "x":False,
                "y":False},
    hover_name="ROOM",
    range_x=[0,100],
    range_y=[0,50],
    title="Room temperatures by 15 minute window for Floor 5",
    text="ROOM"
)
fig.update_xaxes(showgrid=False)
fig.layout.xaxis.title=None
fig.layout.xaxis.showticklabels=False
fig.update_yaxes(showgrid=False)
fig.layout.yaxis.title=None
fig.layout.yaxis.showticklabels=False
fig.update_traces(textfont_size=14, textfont_color="White") #changes the marker labels to bold white
fig.update_traces(marker=dict(symbol="square")) #updates the first frame to square so the animation doesn't look funny
for frame in fig.frames:
    frame.data[0]['marker']['symbol'] = "square" #updates the other frames to square so the animation doesn't look funny
    
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 100 #increases the speed (reduces duration) of frames

import base64
#set a local image as a background
image_filename = 'FLOOR5-3.jpg'
plotly_logo = base64.b64encode(open(image_filename, 'rb').read())

fig.update_layout(
                images= [dict(
                    source='data:image/png;base64,{}'.format(plotly_logo.decode()),
                    xref="paper", yref="paper",
                    x=0, y=1,
                    sizex=1, sizey=1,
                    xanchor="left",
                    yanchor="top",
                    sizing="stretch",
                    layer="below")])
fig.write_html("..\\Interactive System\\floor-exploration\\FLOOR-5.html")
fig.show()